<a href="https://colab.research.google.com/github/GGGGLORIA12138/Optimizing-Conference-Scheduling-/blob/main/730_Final_Project_scheduling.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#set up

In [ ]:
#Copy-and-paste the code below to use as "set-up" when your optimization model uses Pyomo. 
#Uncomment the appropriate solver that you need.
#for reference, see https://colab.research.google.com/drive/1yGk8RB5NXrcx9f1Tb-oCiWzbxh61hZLI?usp=sharing

#installing and importing pyomo
!pip install -q pyomo
from pyomo.environ import *

###installing and importing specific solvers (uncomment the one(s) you need)
###glpk
!apt-get install -y -qq glpk-utils
###cbc
#!apt-get install -y -qq coinor-cbc
###ipopt
#!wget -N -q "https://ampl.com/dl/open/ipopt/ipopt-linux64.zip"
#!unzip -o -q ipopt-linux64
###bonmin
#!wget -N -q "https://ampl.com/dl/open/bonmin/bonmin-linux64.zip"
#!unzip -o -q bonmin-linux64
###couenne
#!wget -N -q "https://ampl.com/dl/open/couenne/couenne-linux64.zip"
#!unzip -o -q couenne-linux64
###geocode
#!wget -N -q "https://ampl.com/dl/open/gecode/gecode-linux64.zip"
#!unzip -o -q gecode-linux64

#Using the solvers:
#SolverFactory('glpk', executable='/usr/bin/glpsol')
#SolverFactory('cbc', executable='/usr/bin/cbc')
#SolverFactory('ipopt', executable='/content/ipopt')
#SolverFactory('bonmin', executable='/content/bonmin')
#SolverFactory('couenne', executable='/content/couenne')
#SolverFactory('gecode', executable='/content/gecode')

In [ ]:
import pandas as pd

#Scheduling Problem


In [ ]:
#Model inputs
day_salary = 186 #Salary for each worker per day. Can be changed if needed 

In [ ]:
#Constraint inputs
min_num_workers = 6 #minimum number of worker on shift per day
min_num_work_days = 5 #minimum number of days a worker need to work per week
max_num_work_days = 6 #maximum number of days a worker can work per week

In [ ]:
num_workers = 8 # number of worker need to be assigned. index by i
num_weeks = 4 # number of weeks need to be scheduled for. index by j
num_days = 7 # number of days in each week. index by k

#defining the optimization model
model = ConcreteModel()

#DVs
model.x = Var(range(num_workers), range(num_weeks), range(num_days), domain= Binary) #Assign workers to which days?

#Objective: minimum total labor cost 
Total_days_worked = sum(sum(sum(model.x[i,j,k] for k in range(7)) for j in range(4)) for i in range(8)) #Total days worked = sum of worked days per worker in the 4 weeks.
Total_labor_cost = day_salary * Total_days_worked

model.Objective = Objective(expr = Total_labor_cost, sense = minimize)

#constraits:

##constraint1 - each day request at least 6 workers
model.minworker = ConstraintList()
for j in range(4):
  for k in range(7):
    model.minworker.add(sum(model.x[i,j,k] for i in range(8)) >= min_num_workers)

##constraint2 - each worker can only have 2 rest days per week
model.mindays = ConstraintList()
for i in range(8):
  for j in range(4):
    model.mindays.add(sum(model.x[i,j,k] for k in range(7)) >= min_num_work_days)

##constraint3 - no worker works more than 6 days per week
model.maxdays = ConstraintList()
for i in range(8):
  for j in range(4):
    model.maxdays.add(sum(model.x[i,j,k] for k in range(7)) <= max_num_work_days)

##constraint4 - each worker cannot have a rest on the same day as the previous week
model.restday = ConstraintList()

for i in range(8):
  for j in range(3):
    for k in range(7):
      model.maxdays.add((model.x[i,j,k] + model.x[i,j+1,k]) >= 1)


In [ ]:
#solve the model
opt = SolverFactory('glpk')
opt.options['mipgap'] = 0 #specifies the optimality gap tolerance (.01 means can stop if <1% of optimal obj)
results = opt.solve(model, tee=True) #can set tee=True if you want to see the details.

GLPSOL: GLPK LP/MIP Solver, v4.65
Parameter(s) specified in the command line:
 --mipgap 0 --write /tmp/tmpnamnrswp.glpk.raw --wglp /tmp/tmpko2tyla_.glpk.glp
 --cpxlp /tmp/tmp5awhdhsb.pyomo.lp
Reading problem data from '/tmp/tmp5awhdhsb.pyomo.lp'...
/tmp/tmp5awhdhsb.pyomo.lp:2249: warning: lower bound of variable 'x1' redefined
/tmp/tmp5awhdhsb.pyomo.lp:2249: warning: upper bound of variable 'x1' redefined
261 rows, 225 columns, 1009 non-zeros
224 integer variables, all of which are binary
2473 lines were read
Writing problem data to '/tmp/tmpko2tyla_.glpk.glp'...
1984 lines were written
GLPK Integer Optimizer, v4.65
261 rows, 225 columns, 1009 non-zeros
224 integer variables, all of which are binary
Preprocessing...
168 hidden packing inequaliti(es) were detected
32 hidden covering inequaliti(es) were detected
260 rows, 224 columns, 1008 non-zeros
224 integer variables, all of which are binary
Scaling...
 A: min|aij| =  1.000e+00  max|aij| =  1.000e+00  ratio =  1.000e+00
Problem data 

In [ ]:
#print the results: Alternative 1
print("Total cost:", model.Objective())
solution = [[[model.x[i,j,k]() for k in range (7)] for j in range (4)] for i in range(8)]


pd.DataFrame(solution)

Total cost: 31248.0


,0,1,2,3
0,"[1.0, 1.0, 0.0, 1.0, 1.0, 1.0, 0.0]","[0.0, 0.0, 1.0, 1.0, 1.0, 1.0, 1.0]","[1.0, 1.0, 1.0, 1.0, 1.0, 0.0, 0.0]","[1.0, 0.0, 0.0, 1.0, 1.0, 1.0, 1.0]"
1,"[1.0, 0.0, 0.0, 1.0, 1.0, 1.0, 1.0]","[1.0, 1.0, 1.0, 1.0, 1.0, 0.0, 1.0]","[1.0, 0.0, 0.0, 1.0, 1.0, 1.0, 1.0]","[0.0, 1.0, 1.0, 0.0, 1.0, 1.0, 1.0]"
2,"[1.0, 0.0, 1.0, 1.0, 1.0, 0.0, 1.0]","[1.0, 1.0, 1.0, 0.0, 0.0, 1.0, 1.0]","[1.0, 0.0, 1.0, 1.0, 1.0, 0.0, 1.0]","[1.0, 1.0, 1.0, 0.0, 0.0, 1.0, 1.0]"
3,"[1.0, 1.0, 1.0, 1.0, 1.0, 0.0, 0.0]","[1.0, 1.0, 1.0, 0.0, 0.0, 1.0, 1.0]","[0.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.0]","[1.0, 1.0, 0.0, 1.0, 0.0, 1.0, 1.0]"
4,"[1.0, 1.0, 1.0, 1.0, 0.0, 1.0, 1.0]","[0.0, 1.0, 0.0, 1.0, 1.0, 1.0, 1.0]","[1.0, 1.0, 1.0, 1.0, 0.0, 1.0, 1.0]","[1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.0]"
5,"[1.0, 1.0, 1.0, 1.0, 0.0, 1.0, 1.0]","[1.0, 1.0, 0.0, 1.0, 1.0, 1.0, 0.0]","[0.0, 1.0, 1.0, 0.0, 1.0, 1.0, 1.0]","[1.0, 1.0, 1.0, 1.0, 1.0, 0.0, 1.0]"
6,"[0.0, 1.0, 1.0, 0.0, 1.0, 1.0, 1.0]","[1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.0]","[1.0, 1.0, 1.0, 0.0, 1.0, 1.0, 1.0]","[0.0, 1.0, 1.0, 1.0, 1.0, 0.0, 1.0]"
7,"[0.0, 1.0, 1.0, 0.0, 1.0, 1.0, 1.0]","[1.0, 0.0, 1.0, 1.0, 1.0, 0.0, 1.0]","[1.0, 1.0, 0.0, 1.0, 0.0, 1.0, 1.0]","[1.0, 0.0, 1.0, 1.0, 1.0, 1.0, 0.0]"


##print the optimized schedule



In [ ]:
worker = ['worker1', 'worker2', 'worker3', 'worker4', 'worker5', 'worker6', 'worker7', 'worker8']
day = ['Monday','Tuesday','Wednesday','Thursday','Friday','Saturday','Sunday']

days_worked_in_4week = []
for i in range(8):
  days_worked_in_4week.append(sum(sum(model.x[i,j,k]() for k in range(7)) for j in range(4)))

Total_days_worked_solved = [' ' for i in range(7)] #Total days worked = sum of worked days per worker in a the 4 weeks.
Total_days_worked_solved.append(sum(sum(sum(model.x[i,j,k]() for k in range(7)) for j in range(4)) for i in range(8)))

Total_labor_cost = [' ' for i in range(7)]
Total_labor_cost.append(model.Objective())


#print the results
print("Total labor cost:", model.Objective())


week1 = [[model.x[i,0,k]() for k in range (7)] for i in range(8)]
week2 = [[model.x[i,1,k]() for k in range (7)] for i in range(8)]
week3 = [[model.x[i,2,k]() for k in range (7)] for i in range(8)]
week4 = [[model.x[i,3,k]() for k in range (7)] for i in range(8)]

df1 = pd.DataFrame(week1, index = worker, columns = day) # Week1 table of schedule
df2 = pd.DataFrame(week2, index = worker, columns = day) # Week2 table of schedule
df3 = pd.DataFrame(week3, index = worker, columns = day) # Week3 table of schedule
df4 = pd.DataFrame(week4, index = worker, columns = day) # Week4 table of schedule

scheduling_solution_df = pd.concat([df1, df2, df3, df4], axis=1) # combining the 4 tables

scheduling_solution_df['Total days worked/worker'] = days_worked_in_4week #adding a new column of "Total days worked/worker"
scheduling_solution_df['Total days worked'] = Total_days_worked_solved #adding a new column of "Total days worked"
scheduling_solution_df['Total labor worked'] = Total_labor_cost #adding a new column of "Total days worked"


scheduling_solution_df

Total labor cost: 31248.0


,Monday,Tuesday,Wednesday,Thursday,Friday,Saturday,Sunday,Monday,Tuesday,Wednesday,...,Monday,Tuesday,Wednesday,Thursday,Friday,Saturday,Sunday,Total days worked/worker,Total days worked,Total labor worked
worker1,1.0,1.0,0.0,1.0,1.0,1.0,0.0,0.0,0.0,1.0,...,1.0,0.0,0.0,1.0,1.0,1.0,1.0,20.0,,
worker2,1.0,0.0,0.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,...,0.0,1.0,1.0,0.0,1.0,1.0,1.0,21.0,,
worker3,1.0,0.0,1.0,1.0,1.0,0.0,1.0,1.0,1.0,1.0,...,1.0,1.0,1.0,0.0,0.0,1.0,1.0,20.0,,
worker4,1.0,1.0,1.0,1.0,1.0,0.0,0.0,1.0,1.0,1.0,...,1.0,1.0,0.0,1.0,0.0,1.0,1.0,20.0,,
worker5,1.0,1.0,1.0,1.0,0.0,1.0,1.0,0.0,1.0,0.0,...,1.0,1.0,1.0,1.0,1.0,1.0,0.0,23.0,,
worker6,1.0,1.0,1.0,1.0,0.0,1.0,1.0,1.0,1.0,0.0,...,1.0,1.0,1.0,1.0,1.0,0.0,1.0,22.0,,
worker7,0.0,1.0,1.0,0.0,1.0,1.0,1.0,1.0,1.0,1.0,...,0.0,1.0,1.0,1.0,1.0,0.0,1.0,22.0,,
worker8,0.0,1.0,1.0,0.0,1.0,1.0,1.0,1.0,0.0,1.0,...,1.0,0.0,1.0,1.0,1.0,1.0,0.0,20.0,168.0,31248.0


In [ ]:
#Convert the dataframe to a CSV file
df = pd.DataFrame(scheduling_solution_df)
df.to_csv('Scheduling_solution_by_Python.csv')